In [2]:

import sys, os


sys.path.append(os.path.join(sys.path[0], '../'))
sys.path.append(os.path.join(sys.path[0], '../../gans/'))

import random
from models.hg import HG_skeleton

import matplotlib
from matplotlib import pyplot as plt
import time
from dataset.toheatmap import ToGaussHeatMap, CoordToGaussSkeleton

from typing import List
import torch.utils.data
from torch import Tensor
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from dataset.lazy_loader import LazyLoader, Celeba
from dataset.probmeasure import UniformMeasure2D01
from gan.loss.stylegan import StyleGANLoss
from gan.models.stylegan import StyleGanModel
from gan.nn.stylegan.discriminator import Discriminator
from gan.nn.stylegan.generator import Generator, FromStyleConditionalGenerator, HeatmapToImage
from gan.noise.stylegan import mixing_noise
from optim.accumulator import Accumulator
from parameters.path import Paths


def send_images_to_tensorboard(writer, data: Tensor, name: str, iter: int, count=8, normalize=True, range=(-1, 1)):
    with torch.no_grad():
        grid = make_grid(
            data[0:count], nrow=count, padding=2, pad_value=0, normalize=normalize, range=range,
            scale_each=False)
        writer.add_image(name, grid, iter)


manualSeed = 71
random.seed(manualSeed)
torch.manual_seed(manualSeed)

batch_size = 8
image_size = 256
noise_size = 512

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

test_sample_z = torch.randn(8, noise_size, device=device)
Celeba.batch_size = batch_size



In [3]:
generator = Generator(FromStyleConditionalGenerator(image_size, noise_size), n_mlp=8)
discriminator = Discriminator(image_size)

starting_model_number = 290000
weights = torch.load(
    f'{Paths.default.models()}/celeba_gan_256_{str(starting_model_number).zfill(6)}.pt',
    map_location="cpu"
)

discriminator.load_state_dict(weights['d'])
generator.load_state_dict(weights['g'])

heatmap2image = HeatmapToImage(generator.gen, generator.z_to_style, 1).cuda()
discriminator = discriminator.cuda()

skeletoner = CoordToGaussSkeleton(256, 4)
hg = HG_skeleton(skeletoner).cuda()
hm_discriminator = Discriminator(image_size, input_nc=1).cuda()


gan_model_tuda = StyleGanModel(heatmap2image, StyleGANLoss(discriminator), (0.001, 0.0015))
gan_model_obratno = StyleGanModel(hg, StyleGANLoss(hm_discriminator), (0.001, 0.0015))


TypeError: __init__() got an unexpected keyword argument 'input_nc'

In [ ]:
batch = next(LazyLoader.w300().loader_train_inf)

real_img = batch["data"].cuda()
landmarks = batch["meta"]["keypts_normalized"].cuda()
measure = UniformMeasure2D01(torch.clamp(landmarks, max=1))
skeleton = skeletoner.forward(measure.coord).sum(1, keepdim=True)
noise = mixing_noise(batch_size, 512, 0.9, device)


In [4]:
fake, _ = heatmap2image.forward(skeleton, noise)

gan_model_tuda.discriminator_train([real_img], [fake.detach()])
gan_model_tuda.generator_loss([real_img], [fake]).minimize_step(gan_model_tuda.optimizer.opt_min)

NameError: name 'heatmap2image' is not defined

In [5]:
sk_pred = hg.forward(real_img)["skeleton"]
gan_model_obratno.discriminator_train([skeleton], [sk_pred.detach()])
gan_model_obratno.generator_loss([skeleton], [sk_pred]).minimize_step(gan_model_obratno.optimizer.opt_min)


NameError: name 'hg' is not defined